Click the button below if you want to see the code behind the widgets. If you do click  and want to go back to the 'no-code' view, scroll down until you find the 'hide code' button.

In [1]:
"""
MIT License

Copyright (c) 2020 Sylvain Barde - University of Kent

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams
from IPython.display import display, HTML

javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Click to show code", True: "Click to hide code"}

def toggle_code(state):
    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)

    display(HTML(output))
    
def button_action(value):
    """
    Calls the toggle_code function and updates the button description.
    """
    state = value.new

    toggle_code(state)

    value.owner.description = button_descriptions[state]

def profit_widget(Qmax_init = 8,  a_d_init = 2, b_d_init = 5, c_d_init = 50, 
                   a_s_init = 1, b_s_init = -4, c_s_init = 10, d_s_init = 0, 
                   revFlag_init = True, cstFlag_init = True):

    # Declare widgets for interactive input
    Qmax_slider = widgets.IntSlider(min= 5,
                                 max=50,
                                 step=1,
                                 description=r'Maximum $Q$:',
                                 value = Qmax_init,
                                 continuous_update =False)
    a_d_slider = widgets.FloatSlider(min=0,
                                 max=10,
                                 description= r'$a_d$:',
                                 value = a_d_init,
                                 continuous_update =False)
    b_d_slider = widgets.FloatSlider(min=-50,
                                 max=50,
                                 description= r'$b_d$:',
                                 value=b_d_init,
                                 continuous_update =False)
    c_d_slider = widgets.FloatSlider(min=-100,
                                 max=100,
                                 description= r'$c_d$:',
                                 value = c_d_init,
                                 continuous_update =False)
    a_s_slider = widgets.FloatSlider(min=0,
                                 max=10,
                                 description= r'$a_s$:',
                                 value = a_s_init,
                                 continuous_update =False)
    b_s_slider = widgets.FloatSlider(min=-50,
                                 max=50,
                                 description= r'$b_s$:',
                                 value=b_s_init,
                                 continuous_update =False)
    c_s_slider = widgets.FloatSlider(min=-100,
                                 max=100,
                                 description= r'$c_s$:',
                                 value = c_s_init,
                                 continuous_update =False)
    d_s_slider = widgets.FloatSlider(min=-100,
                                 max=100,
                                 description= r'$d_s$:',
                                 value = d_s_init,
                                 continuous_update =False)
    revFlag_check = widgets.Checkbox(value = revFlag_init,
                                   description='Show revenue',
                                   disabled=False,
                                   indent=True)
    cstFlag_check = widgets.Checkbox(value = cstFlag_init,
                                   description='Show costs',
                                   disabled=False,
                                   indent=True)
    
    # Link widgets as required


    def profit_plot(Qmax, a_d, b_d, c_d, a_s, b_s, c_s, d_s, revFlag, cstFlag):

        # create a quantity vector
        Q = np.arange(-1e-3,Qmax,(Qmax-1e-3)/500)

        # Calculate Revenue vectors
        TR = lambda Q: -a_d*Q**3 + b_d*Q**2 + c_d*Q
        AR = lambda Q: -a_d*Q**2 + b_d*Q + c_d
        mR = lambda Q: -3*a_d*Q**2 + 2*b_d*Q + c_d

        # Calculate Cost vectors
        TC = lambda Q: a_s*Q**3 + b_s*Q**2 + c_s*Q + d_s
        AC = lambda Q: a_s*Q**2 + b_s*Q + c_s + d_s/Q
        mC = lambda Q: 3*a_s*Q**2 + 2*b_s*Q + c_s

        # Function labels
        sig = []
        for param in [b_d, c_d, b_s, c_s, d_s]:
            if param >= 0:
                sig.append('+')
            else:
                sig.append('-')

        lblTR = r'$TR = -{:.0f}Q^3 {:s} {:.0f}Q^2 {:s} {:.0f}Q$'
        lblAR = r'$AR = -{:.0f}Q^2 {:s} {:.0f}Q {:s} {:.0f}$'
        lblmR = r'$mR = -{:.0f}Q^2 {:s} {:.0f}Q {:s} {:.0f}$'

        lblTC = r'$TC = {:.0f}Q^3 {:s} {:.0f}Q^2 {:s} {:.0f}Q {:s} {:.0f}$'
        lblAC = r'$AC = {:.0f}Q^2 {:s} {:.0f}Q {:s} {:.0f} {:s} {:.0f}/Q$'
        lblmC = r'$mC = {:.0f}Q^2 {:s} {:.0f}Q {:s} {:.0f}$'

        # find profit-maximising point
        Qval = Q[np.argmax(TR(Q)-TC(Q))]
        PiMax = TR(Qval) - TC(Qval)
        Pmax = max(max(AR(Q)),max(AC(Q)),max(mR(Q)),max(mC(Q)))
        TRmax = max(max(TR(Q)),max(TC(Q)))

        # Create figure
        mrkrSize = 2*rcParams['lines.markersize'] ** 2
        fig, ax = plt.subplots(nrows=1, ncols=2,figsize=(20,10))

        # Plot revenue if requested
        if revFlag is True:
            ax[0].plot(Q, AR(Q),'b', linewidth=2, alpha=0.6,
              label=lblAR.format(
                      a_d, sig[0], abs(b_d), sig[1], abs(c_d) ))
            ax[0].plot(Q, mR(Q),'b--', linewidth=2, alpha=0.6,
              label=lblmR.format(
                      3*a_d, sig[0], abs(2*b_d), sig[1], abs(c_d) ))

        # Plot Cost if requested
        if cstFlag is True:
            ax[0].plot(Q, AC(Q),'r', linewidth=2, alpha=0.6,
              label = lblAC.format(
                      a_s, sig[2], abs(b_s), sig[3], abs(c_s), sig[4], abs(d_s)))
            ax[0].plot(Q, mC(Q),'r--', linewidth=2, alpha=0.6,
              label=lblmC.format(
                      3*a_s, sig[2], abs(2*b_s), sig[3], abs(c_s) ))

        if revFlag is True and cstFlag is True:
            # Plot profits
            ax[0].fill([0,Qval,Qval,0],[AC(Qval),AC(Qval),AR(Qval),AR(Qval)],'g',alpha = 0.2,
                        label = r'$\Pi = (AR-AC) \times Q$')
            ax[0].scatter([Qval, Qval, Qval],
                          [AR(Qval),mR(Qval),AC(Qval)],
                          s=mrkrSize, c='k', alpha=0.6, label='Max profits')
            ax[0].plot([Qval,Qval],[0,AR(Qval)],'k--',linewidth=1)    
            ax[0].plot([0,Qval],[AR(Qval),AR(Qval)],'k--',linewidth=1)
            ax[0].plot([0,Qval],[AC(Qval),AC(Qval)],'k--',linewidth=1)

        # Add legend and format axes to look nice
        if revFlag is True or cstFlag is True:
            ax[0].legend(loc='upper center', frameon=False,prop={'size':20})
        ax[0].autoscale(enable=True, axis='both', tight=True)
        ax[0].set_ylim(top = Pmax, bottom = 0)
        ax[0].set_xlim(right = Qmax, left = 0)
        ax[0].spines['top'].set_visible(False)
        ax[0].spines['right'].set_visible(False)
        ax[0].set_xlabel(r'$Q$', fontdict = {'fontsize': 25},position=(1, 0))
        ax[0].set_ylabel(r'$£$', fontdict = {'fontsize': 25},position=(0, 1), rotation=0)
        ax[0].plot(1, 0, ">k", transform=ax[0].get_yaxis_transform(), clip_on=False)
        ax[0].plot(0, 1, "^k", transform=ax[0].get_xaxis_transform(), clip_on=False)
        ax[0].tick_params(labelsize=20)

        # Plot Total revenue
        if revFlag is True:
            ax[1].plot(Q, TR(Q),'b', linewidth=2, alpha=0.6,
              label=lblTR.format(
                      a_d, sig[0], abs(b_d), sig[1], abs(c_d) ))

        # Plot Cost if requested
        if cstFlag is True:
            ax[1].plot(Q, TC(Q),'r', linewidth=2, alpha=0.6,
              label = lblTC.format(
                      a_s, sig[2], abs(b_s), sig[3], abs(c_s), sig[4], abs(d_s)))

        if revFlag is True and cstFlag is True:
            # Plot profits
            ax[1].plot(Q, TR(Q) - TC(Q),'g', linewidth=2, alpha=0.6,
                        label=r'$\Pi = TR -TC$')

            # Add markers for the optimal points, with dotted lines
            ax[1].scatter([Qval,Qval,Qval], 
                         [TR(Qval),TC(Qval),PiMax], 
                         s=mrkrSize, c='k', alpha=0.6, label='Max profits')
            ax[1].plot([Qval,Qval],[0,TR(Qval)],'k--',linewidth=1)
            ax[1].plot([Qval,Qval],[0,TR(Qval)],'k--',linewidth=1)

            # Add parallel slope lines
            dQ = Qmax*0.1
            ax[1].plot([Qval-dQ,Qval+dQ],
                      [PiMax,PiMax],
                      'k--',linewidth=1)
            ax[1].plot([Qval-dQ,Qval+dQ],
                      [TC(Qval)-dQ*mC(Qval),TC(Qval)+dQ*mC(Qval)],
                      'k--',linewidth=1)
            ax[1].plot([Qval-dQ,Qval+dQ],
                      [TR(Qval)-dQ*mR(Qval),TR(Qval)+dQ*mR(Qval)],
                      'k--',linewidth=1)

        # Add legend and format axes to look nice
        if revFlag is True or cstFlag is True:
            ax[1].legend(loc='upper center', frameon=False,prop={'size':20})
        ax[1].autoscale(enable=True, axis='both', tight=True)
        ax[1].set_ylim(top = TRmax, bottom = 0)
        ax[1].set_xlim(right = Qmax, left = 0)
        ax[1].spines['top'].set_visible(False)
        ax[1].spines['right'].set_visible(False)
        ax[1].set_xlabel(r'$Q$', fontdict = {'fontsize': 25},position=(1, 0))
        ax[1].set_ylabel(r'$£$', fontdict = {'fontsize': 25},position=(0, 1), rotation=0)
        ax[1].plot(1, 0, ">k", transform=ax[1].get_yaxis_transform(), clip_on=False)
        ax[1].plot(0, 1, "^k", transform=ax[1].get_xaxis_transform(), clip_on=False)
        ax[1].tick_params(labelsize=20)  

        plt.tight_layout()
    
    out = widgets.interactive_output(profit_plot, {'Qmax': Qmax_slider,
                                                   'a_d': a_d_slider,
                                                   'b_d': b_d_slider,
                                                   'c_d': c_d_slider,
                                                   'a_s': a_s_slider,
                                                   'b_s': b_s_slider,
                                                   'c_s': c_s_slider,
                                                   'd_s': d_s_slider,
                                               'revFlag': revFlag_check,
                                               'cstFlag': cstFlag_check})
    
    output = widgets.VBox([out,
                  widgets.HBox([Qmax_slider,
                               revFlag_check,
                               cstFlag_check]),
                  widgets.HBox([a_d_slider, 
                                b_d_slider,
                                c_d_slider]),
                  widgets.HBox([a_s_slider, 
                                b_s_slider,
                                c_s_slider,
                                d_s_slider])])
    display(output)    
    

In [2]:
state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")

display(button)

ToggleButton(value=False, description='Click to show code')

### Maximisation at the margin: using derivatives in economics

Finding the maximum (or minimum) of a function is a common task in economics, as it is standard to assume that the behaviour of individual agents (firms, households, policy makers, etc.) is to find the maximum or minimum of a particular quantity that they care about. So for firms, this will typically be profits, while households are assumed to maximise their satisfaction. Policy makers might want to minimise inflation, or maximise social welfare, etc.

This purpose of this particular notebook is therefore to illustrate how this is done in a very simple profit-maximisation example. This will also give us the opportunity to go over the specific notation used in economics, as well as some of the thinking behind the use of derivatives in economics.

As I just said, we are going to look at a profit maximisation problem, and in order to make things simple, we are going to assume a monopoly, i.e. there is a single firm supplying a market. This means that the demand curve facing the firm is simply the demand curve for the entire market. So the goal of the monopolist is to maximise its profits $\Pi$, which is simply the difference between the total revenue achieved by selling a quantity $Q$ and the total cost of producing that quantity.

$$\Pi \; = \; TR(Q) \; - \; TC(Q)$$

In order to calculate the profit function, we therefore need a total revenue function and a total cost function. Let's start by looking at revenue.


#### The revenue side

We've already looked at one example of total revenue ($TR$) and average revenue ($AR$) in the notebook that covered applications of quadratic functions in economics. In that notebook, we saw that the U-shaped total revenue curve emerged as the product of two linear factors, quantity $Q$ and the (linear) demand curve, which also happens to be the average revenue ($AR$) curve.

The example below extends this a bit but the general picture stays the same. The $AR$ curve is actually the inverse demand curve on the market, and total revenue is simply obtained by multiplying it by quantity Q. The specific equation I've picked for the $AR$ curve is:

$$AR \; = \; -2Q^2 + 5Q +50$$

The widget will allow you to play around with whatever numbers you want, but I'll stick with these for my explanations. If we multiply this by $Q$ we get the $TR$ curve, which is displayed in the right hand side diagram. We can assume here that quantity is measured in thousands of units.

$$TR \; = \;AR\;\times\;Q\;=\; -2Q^3 + 5Q^2 +50Q$$

The first way in which this diagram is different than the one in the total revenue notebook you saw before is that total revenue is now a cubic equation (we have a $Q^3$ term), and no longer a quadratic. We still retain the overall inverse U-shape of total revenue, but this allows more flexibility. In this example total revenue grows relatively slowly initially, but falls very fast after the maximum.

The second way in which the diagram is different is that we have an extra curve! This is labeled $mR$ for **marginal revenue**. The economic idea behind marginal revenue is that it is the extra revenue generated by selling one more unit of the good. From a mathematical point of view, as long as the original quantity is large (say thousands of units), so that this one-unit increase is comparatively small, then marginal revenue is essentially the first derivative of total revenue:

$$mR \; = \; \frac{{\rm d}TR}{{\rm d}Q} \;=\; -6Q^2 + 10Q + 50$$

In [6]:
profit_widget(cstFlag_init = False)

#### The cost side

Now that we have the revenue side sorted out, let's turn to the production costs. The various curves used are shown in the diagram below. Here, we start with the total production costs $TC$, for which I've chosen the following function:

$$TC \; =\; Q^3 - 4Q^2 +10Q$$

Again, the numbers in the widgets can be moved around, but these work well for the purposes of the illustration. The $TC$ curve, shown on the right-hand side shows costs that grow slowly to start with but pick up sharply past a certain point. This can be explained by looking at the average cost of production $AC$, visible on the left. The equation for $AC$ can be obtained by dividing total cost by the quantity produced:

$$AC \; = \frac{TC}{Q} \; =\; Q^2 - 4Q + 10$$

As you can see, the average costs $AC$ initially fall as the quantity produced increases, which is why the growth in total costs is initially so slow: while production costs will increase as you increase production, this is partially offset by the fact that the cost per unit is falling. This U-shaped feature of the $AC$ curve is very common: setting up a production plant has some big up-front costs: you may need to have a factory, purchase machinery, do some R&D to develop an attractive and competitive product, protect your intellectual property by paying for a patent, etc. All these expenses only need to happen once, so the more your produce, the more you can spread these costs over the volume of production. This is known as increasing returns to scale and explains why a product that is mass-produced is cheaper than a product manufactured in small volumes. 

This only last so long, however: past a certain point, increasing production will lead to increasing average costs, and the total cost curve will become very steep. As for the revenue diagrams above, we can also calculate the **marginal cost** of production $mC$ which is the increase in total costs resulting from the production of one more unit. As above, this is the first derivative of Total Cost with respect to quantity $Q$.

$$mC \; = \frac{{\rm d}TC}{{\rm d}Q} \; =\; 3Q^2 - 8Q + 10$$

In [4]:
profit_widget(revFlag_init = False)

Now that we have the revenue and cost information we need, we have all we need to find the level of output $Q$ that will maximise profits for this monopolist. There are two ways of doing this, which are equivalent and lead to the same solution. One of them, however, illustrates the way economists often look at these problems.

#### Finding the profit-maximising quantity using the profit function

The first approach is to find the profit function, then differentiate it to get the first and second order conditions. We know that profits is simply the difference between total revenue and total cost.

$$\Pi \; = \; TR \; - \; TC$$

By replacing $TR$ and $TC$ by the values above, we can obtain the equation for the profit function:

$$\begin{aligned}
\Pi & \; = \; -2Q^3 + 5Q^2 +50Q - \left( Q^3 - 4Q^2 +10Q\right) \\
\Pi & \; = \; -2Q^3 + 5Q^2 +50Q -  Q^3 + 4Q^2 - 10Q \\
\Pi & \; = \; -3Q^3 + 9Q^2 + 40Q\\
\end{aligned}$$

This function is shown in green in the right-hand side diagram. To find the quantity that maximises profits, we need to take the derivative with respect to $Q$, and set it for zero:

$$\frac{{\rm d}\Pi}{{\rm d}Q} \; =\; -9Q^2 + 18Q + 40 \; =\; 0$$

Finding the roots of the quadratic will give us the turning points, one of which (we hope) is a maximum

$$-9Q^2 + 18Q + 40 \; =\; 0$$

The discriminant of the quadratic is $18^2 + 4 \; \times \; 9 \; \times \;40 = 1764 = 42^2$, so we have two solutions:

$$\left\{
\begin{aligned}
Q_1 &\; = \; \frac{18-42}{18} = \frac{-24}{18} = \frac{-4}{3} \\
Q_2 &\; = \; \frac{18+42}{18} = \frac{60}{18} = \frac{10}{3} \\
\end{aligned}\right.$$

Clearly, $Q_1$ is not a feasible solution (you can't produce a negative quantity), so our candidate maximum is $\frac{10}{3}\approx 3.333$. Note that at this point if we didn't have the diagram we wouldn't necessarily know if it is a maximum, so we'd have to check the second order condition. The second derivative of the profit function is given by:

$$\frac{{\rm d^2}\Pi}{{\rm d}Q^2} \; =\; -18Q + 18 \; =\; 18(1-Q)$$

Clearly, given that $Q_2>1$, this derivative is negative at that point, so we know that the profit function has a local maximum at that point. This is what we see in the right hand side diagram: the slope of the profit function, in green, is flat for $Q=3.333\dots$, and we have found our maximum.


In [5]:
profit_widget()

#### Finding the profit-maximising quantity using marginal revenue and cost

The other way to maximise profits is to look at what happens to the marginal variables ($mR$ and $mC$) This carries much more economic meaning, and is often the way this is explained in textbooks. To do this, let's go back to the basic profit equation.

$$\Pi \; = \; TR \; - \; TC$$

To maximise profits, we need to take the derivative and set it to zero (this is our first order condition). Because we know that the derivative of a sum is just the sum of the derivatives, we can do this ***without replacing  the equations for $TR$ and $TC$***:

$$\begin{aligned}
\frac{{\rm d}\Pi}{{\rm d}Q} & \; =\; \frac{{\rm d}TR}{{\rm d}Q} - \frac{{\rm d}TC}{{\rm d}Q} \; =\; 0 \\
\frac{{\rm d}\Pi}{{\rm d}Q} & \; = \; mR-mC  =\; 0 \\
\end{aligned}$$

This directly tells us that profit is maximised when:

$$\begin{aligned}
mR-mC & =\; 0 \\
mR & =\; mC \\
\end{aligned}$$

If we replace the equations for $mR$ and $mC$ that we found in the sections above, we get the following expression:

$$-6Q^2 + 10Q + 50 \quad = \quad  3Q^2 - 8Q + 10$$

Tidy this expression up by grouping terms on the left and you get:

$$-9Q^2 + 18Q + 40 = 0$$

This is the **same** quadratic equation as we got when we directly took the derivative of the profit function. Unsurprisingly, we will therefore find the same value for the profit-maximising quantity.

But let's go back one step and note that the $mR = mC$ first order condition will be true ***regardless of the equations you pick for revenue and cost***. This explains why:
- In the left hand side diagram we see that the $mR$ and $mC$ curves cross exactly at the profit maximising quantity of $3.333\dots$.
- Equivalently, in the diagram on the right, we see that at the profit-maximising quantity the slope of the total revenue function is equal to the slope of the total cost function (the two black dotted lines are parallel).

This **marginal** approach to finding the maximum profits has several advantages:
1. It saves a bit ot time, as you don't need to calculate the profit function then take the first order condition. If you already havd $mC$ and $mR$ (which we did), you can go straight ahead and maximise profits.
2. It offers a nice economic intuition as to **why** profits are maximised at that point: essentially, when you are at the profit maximising quantity, the extra revenue generated by producing an extra unit exactly covers the extra cost of producing it. This means that it is not worth producing that extra unit, so you stop your production run at that point. Checking if the revenue from extra production covers the costs of doing so is something that firms can do empirically, without needed to take derivatives and solving first order conditions!
3. It simplifies the process of checking the second order condition for a maximum. Let's derive the second order condition from the profit function:

$$\begin{aligned}
\frac{{\rm d^2}\Pi}{{\rm d}Q^2} & \; =\; \frac{{\rm d^2}TR}{{\rm d}Q^2} - \frac{{\rm d^2}TC}{{\rm d}Q^2} \\
\frac{{\rm d^2}\Pi}{{\rm d}Q^2} & \; =\; \frac{{\rm d}mR}{{\rm d}Q} - \frac{{\rm d}mC}{{\rm d}Q}\\
\end{aligned}$$

Remember that for a **maximum** we need the SOC to be negative, in other words:

$$\begin{aligned}
\frac{{\rm d}mR}{{\rm d}Q} - \frac{{\rm d}mC}{{\rm d}Q} &< 0\\
\frac{{\rm d}mR}{{\rm d}Q}  &< \frac{{\rm d}mC}{{\rm d}Q}\\
\end{aligned}$$

What this means is that for us to be at a maximum in the profit function, we need to have $mR = mC $ and the slope of the $mR$ curve must be smaller than the slope of the $mC$ curve. If you look at the diagram, we see that the slope of the $mR$ curve is negative, and the slope of the $mC$ curve is positive, so we're fine! In fact, we know that empirically marginal costs tend to increase with quantity wile marginal revenues tend to fall. So this tells us we're pretty much guaranteed a maximum.


#### Wrapping up...

Economists use derivatives all the time to find the maximum (or minimum) of functions we care about. It is important to understand, however, that we have developed a specific way of thinking of derivatives, which are typically called **marginal** values. Crucially, marginal values (costs and revenues in our examples) are more than just a replacement name for the same concept. As we have seen, thinking in terms of variables at the margin allows us to:
- Simplify the process of finding the location of the maximum or minimum: we don't need to calculate the profit function and take its derivative (we work directly with the marginal quantities) and we don't need to explicitly take a second derivative to check we have a maximum (we can just compare the slopes of the marginal functions). This is why, for instance, economists constantly talk about optimisation but nearly never check the second order condition!
- Get some intuition for how an agent might go about finding a maximum in practice (or something very close) without ever actually doing the maths. In our example, the firm just needs to check if the revenue from a bit more production covers the costs of that production, and stops when the two are equal. It's the same thing with you when you go shopping: you don't solve a big maximisation problem in your head, instead you compare the satisfaction you might get from various options.

So while you can absolutely find the maximum profit the 'maths' way (this was the first way we did it) and get the correct result, doing it with marginal values is usually more informative from the point of view of economics.